In [ ]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
key='YOUR KEY'
url = f'https://open.jejudatahub.net/api/proxy/1aD5taat1attaa51Db1511b51ab9Da19/{key}'

In [ ]:
import requests
import json

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_jeju_data(start_date_str, end_date_str, key):
    """
    제주 데이터 허브 API에서 기간별 데이터를 가져오는 함수.

    Args:
        start_date_str: 시작 날짜 문자열 (YYYYMMDD 형식).
        end_date_str: 종료 날짜 문자열 (YYYYMMDD 형식).
        api_key: API 키.

    Returns:
        pandas DataFrame: 모든 날짜의 데이터를 담은 DataFrame.
        API 호출 실패 시 None 반환.
    """

    base_url = 'https://open.jejudatahub.net/api/proxy/1aD5taat1attaa51Db1511b51ab9Da19/'
    all_data = []

    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
    except ValueError:
        print("날짜 형식이 잘못되었습니다. YYYYMMDD 형식으로 입력해주세요.")
        return None

    current_date = start_date
    while current_date <= end_date:
        search_date = current_date.strftime('%Y%m%d')
        url = f'{base_url}{key}?searchDate={search_date}'

        try:
            response = requests.get(url)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

            data = response.json()
            # API 응답 구조에 따라 데이터 추출 방식 조정 필요
            # 예: data['response']['body']['items']['item']
            if 'data' in data and isinstance(data['data'], list):
                items = data['data']
                for item in items:
                    item['searchDate'] = search_date #날짜 정보
                    all_data.append(item)
            else:
                print(f"{search_date} 데이터 없음 또는 API 응답 형식이 예상과 다름: {data}")

        except requests.exceptions.RequestException as e:
            print(f"{search_date} API 요청 실패: {e}")
            return None
        except (KeyError, TypeError) as e:
            print(f"{search_date} 데이터 처리 중 오류 발생: {e}, 응답 데이터: {data}")
            return None

        current_date += timedelta(days=1)

    if all_data:
        df = pd.DataFrame(all_data)
        return df
    else:
        return None


In [ ]:
import boto3
import pandas as pd
from io import StringIO

In [ ]:
# S3 클라이언트 생성
def upload_to_s3(df, bucket_name, s3_key, aws_access_key_id, aws_secret_access_key, aws_region):
    try:
        # S3 클라이언트 생성
        s3_client = boto3.client(
            's3',
            aws_access_key_id='AWS ACCESS KEY ID',
            aws_secret_access_key='AWS SECRET ACCESS KEY',
            region_name='YOUR REGION'
        )

        # DataFrame을 CSV로 변환하여 S3에 업로드
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, encoding='utf-8-sig', index=False)
        csv_buffer.seek(0)

        # S3에 업로드
        s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())
        print(f"S3에 파일 저장 완료: s3://{bucket_name}/{s3_key}")
    
    except Exception as e:
        print(f"S3 업로드 실패: {e}")


In [ ]:
import boto3
import pandas as pd
from io import StringIO

# S3에 CSV 파일 업로드 함수
def upload_to_s3(df, bucket_name, s3_key, aws_access_key_id, aws_secret_access_key, aws_region):
    try:
        # S3 클라이언트 생성
        s3_client = boto3.client(
            's3',
            aws_access_key_id='AWS ACCESS KEY ',
            aws_secret_access_key='AWS SECRET ACCESS KEY',
            region_name='YOUR REGION'
        )

        # DataFrame을 CSV 형식으로 변환하고 메모리 버퍼에 저장
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, encoding='utf-8-sig', index=False)
        csv_buffer.seek(0)  # 버퍼의 시작으로 포인터 이동

        # S3에 CSV 파일 업로드
        s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())
        print(f"S3에 파일 저장 완료: s3://{bucket_name}/{s3_key}")
    
    except Exception as e:
        print(f"S3 업로드 실패: {e}")

In [ ]:
from datetime import datetime
start_date = '20240101'
end_date = datetime.now().strftime('%Y%m%d') # 현재 날짜까지  설정. 전체 기간으로 변경 가능

# 데이터 가져오기
df = fetch_jeju_data(start_date, end_date, key)


if df is not None:
    print("데이터 수집 완료:")
    print(df.head())  # 처음 몇 행 출력
    
    # S3에 업로드할 파일 경로 지정
    s3_bucket_name = 'ip-jeju-airflow'  # S3 버킷 이름
    s3_key = 'data/raw/jeju-daily-weather/'  # S3에 저장할 파일 경로

    # S3에 데이터 업로드
    upload_to_s3(df, s3_bucket_name, s3_key, 
                 aws_access_key_id='AWS ACCESS KEY ID', 
                 aws_secret_access_key='AWS SECRET ACCESS KEY',
                 aws_region='YOUR REGION')
else:
    print("데이터 수집 실패")